In [9]:
import random
import time
import os
from gensim import corpora, matutils
import MeCab
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from matplotlib import pyplot as plt
import sys
import gc

dictionary = corpora.Dictionary.load_from_text('combined_0.80以上の広告なし辞書.txtスコア0.8以上の広告あり辞書.txt.txt')
mecab = MeCab.Tagger('-d  /Users/g_takahashi/genki_local/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
# def tokenize(text):
#     node = mecab.parseToNode(text)
#     while node:
#         if node.feature.split(',')[0] in ['名詞', '形容詞']:
#             yield node.surface.lower()
#         node = node.next

# def get_words(contents):
#     ret = []
#     for k,content in contents.items():
#         ret.append(get_words_main(content))
#     return ret
    
# def get_words_main(content):
#     return [token for token in tokenize(content)]

In [10]:
# ベイジアンネットワークを比較
# 多項分布と仮定した場合.
NB_multinominal = naive_bayes.MultinomialNB(alpha=1.0)
# ベルヌーイ分布
NB_bernoulli = naive_bayes.BernoulliNB(alpha=1.0)

In [11]:
def cross_val(model):
    start = time.time()
    scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=5)
    elapsed_time = time.time() - start
    print("{0} has mean:{1} and std:{2}".format(model,scores.mean(),scores.std()))
    print("elapsed_time: {0}".format(elapsed_time))

In [12]:
# aucをモデルごとに比較する.
def check_auc(model):
    start = time.time()
    model.fit(X_train,y_train.values.ravel())
    # predicted_y = model.predict_proba(X_test)
    predicted_y = model.predict(X_test)
    # auc = roc_auc_score(y_test, predicted_y[:,1]) #二列目を取得する.
    auc = roc_auc_score(y_test, predicted_y)
    elapsed_time = time.time() - start
    print("{0} has {1} auc.".format(model, auc))
    print("elapsed_time: {0}".format(elapsed_time))
# for model in [LR_by_SGD_with_L1, LR_by_SGD_with_L2]:
#     check_auc(model)

In [4]:
# データを読み込む
X_train = pd.DataFrame.from_csv('advertisement_X_train.csv', sep=',')
y_train = pd.DataFrame.from_csv('advertisement_y_train.csv', sep=',')
X_test = pd.DataFrame.from_csv('advertisement_X_test.csv', sep=',')
y_test = pd.DataFrame.from_csv('advertisement_y_test.csv', sep=',')
# X, y が必要な場合
X = pd.concat([X_train,X_test],ignore_index=True)
y = pd.concat([y_train,y_test],ignore_index=True)

In [13]:
for model in [NB_multinominal, NB_bernoulli]:
    cross_val(model)
    check_auc(model)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) has mean:0.7009795515343664 and std:0.0014056100383838565
elapsed_time: 47.73579812049866
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) has 0.6513775052055607 auc.
elapsed_time: 3.454374074935913
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) has mean:0.6830552210407169 and std:0.0020191725712762868
elapsed_time: 396.71151518821716
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) has 0.6322155812706496 auc.
elapsed_time: 85.65788078308105
